https://www.unitstatistics.com/starcraft2/ used for getting base unit Hp, Dps etc.

Based on the data the following information was used for the units:

| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |




In [45]:
from IPython.display import Markdown

table = """
| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |
"""

display(Markdown(table))


| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |


In [46]:
class CurrentBoard:
    def __init__(self, width=30, height=10, marine_count=4, zergling_count=6):
        self.width = width
        self.height = height
        self.board = [[None for _ in range(width)] for _ in range(height)]
        self.marines = []  # List to store Marine objects
        self.zerglings = []  # List to store Zergling objects
        self.center_x = width // 2
        self.center_y = height // 2
        self.marine_count = marine_count
        self.zergling_count = zergling_count

    def is_in_melee_combat(self, unit):
        # Check if unit is adjacent to an enemy unit
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Check adjacent squares (orthogonal directions)
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for dx, dy in directions:
            check_x = unit.x + dx
            check_y = unit.y + dy

            # Check if the adjacent square is within board bounds
            if (0 <= check_x < self.width and
                    0 <= check_y < self.height):
                adjacent_unit = self.board[check_y][check_x]

                # If there's an enemy unit in an adjacent square
                if (adjacent_unit is not None and
                        adjacent_unit.team == enemy_team):
                    return True

        return False

    def get_attackable_targets(self, unit):
        targets = []
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Determine targets based on unit type and attack range
        enemy_list = self.zerglings if unit.team == 'Marine' else self.marines

        for potential_target in enemy_list:
            # Calculate Manhattan distance
            distance = abs(unit.x - potential_target.x) + abs(unit.y - potential_target.y)

            # Marine attacks at range, Zerglings attack in melee
            if unit.team == 'Marine':
                if distance <= unit.attack_range:
                    targets.append(potential_target)
            else:  # Zerglings
                if distance == 1:  # Melee range
                    targets.append(potential_target)

        return targets

    def get_possible_actions(self, unit):
        actions = []

        # Check if unit is in melee combat
        if self.is_in_melee_combat(unit):
            # Can only attack, no movement allowed
            return [('attack', target) for target in self.get_attackable_targets(unit)]

        # If not in melee, can either move or attack
        possible_moves = self.get_possible_moves(unit)
        attack_targets = self.get_attackable_targets(unit)

        # Add move actions
        for move in possible_moves:
            actions.append(('move', move))

        # Add attack actions
        for target in attack_targets:
            actions.append(('attack', target))

        return actions


    def get_possible_moves(self, unit):
        """
        Generate all possible moves for a unit
        Moves are full speed in any of 8 directions
        """
        moves = []

        # 8 possible movement directions (including diagonals)
        directions = [
            (0, 1),  # Up
            (0, -1),  # Down
            (1, 0),  # Right
            (-1, 0),  # Left
            (1, 1),  # Up-Right
            (1, -1),  # Down-Right
            (-1, 1),  # Up-Left
            (-1, -1)  # Down-Left
        ]

        for dx, dy in directions:
            # Calculate end position based on unit's full speed
            move_x = unit.x + dx * unit.speed
            move_y = unit.y + dy * unit.speed

            # Check if move is within board boundaries
            if (0 <= move_x < self.width and
                    0 <= move_y < self.height):
                # Check if destination is not occupied
                if self.board[move_y][move_x] is None:
                    moves.append((move_x, move_y))

        return moves



    def setup_initial_board(self):
        """
        Set up initial board with units at the exact map edges
        """
        # Clear existing units
        self.marines.clear()
        self.zerglings.clear()
        self.board = [[None for _ in range(self.width)] for _ in range(self.height)]

        # Calculate vertical center of the board
        center_y = self.height // 2

        # Calculate front and back row counts
        zergling_front_count = self.zergling_count // 2
        zergling_back_count = self.zergling_count - zergling_front_count

        marine_front_count = self.marine_count // 2
        marine_back_count = self.marine_count - marine_front_count

        # Zerglings formation (left edge)
        front_zergling_x = 0  # Leftmost edge of the map
        back_zergling_x = 1  # One step from the left edge

        # Marines formation (right edge)
        front_marine_x = self.width - 1  # Rightmost edge of the map
        back_marine_x = self.width - 2  # One step from the right edge

        # Vertical positioning for Zerglings
        zergling_front_rows = [
            y for y in range(
                center_y - zergling_front_count // 2,
                center_y + (zergling_front_count + 1) // 2
            )
        ]
        zergling_back_rows = [
            y for y in range(
                center_y - zergling_back_count // 2,
                center_y + (zergling_back_count + 1) // 2
            )
        ]

        # Vertical positioning for Marines
        marine_front_rows = [
            y for y in range(
                center_y - marine_front_count // 2,
                center_y + (marine_front_count + 1) // 2
            )
        ]
        marine_back_rows = [
            y for y in range(
                center_y - marine_back_count // 2,
                center_y + (marine_back_count + 1) // 2
            )
        ]

        # Place Zerglings in front rows
        for y in zergling_front_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(front_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][front_zergling_x] = zergling

        # Place Zerglings in back rows
        for y in zergling_back_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(back_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][back_zergling_x] = zergling

        # Place Marines in front rows
        for y in marine_front_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(front_marine_x, y)
                self.marines.append(marine)
                self.board[y][front_marine_x] = marine

        # Place Marines in back rows
        for y in marine_back_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(back_marine_x, y)
                self.marines.append(marine)
                self.board[y][back_marine_x] = marine

        # Verify unit counts
        assert len(
            self.zerglings) == self.zergling_count, f"Zergling count is {len(self.zerglings)}, should be {self.zergling_count}"
        assert len(self.marines) == self.marine_count, f"Marine count is {len(self.marines)}, should be {self.marine_count}"






    def display_board(self):
        """
        Visualize the current board state with precise unit placement
        """
        # Create a copy of the board to visualize
        display_grid = [[' .' for _ in range(self.width)] for _ in range(self.height)]

        # Print the board with detailed unit placement
        print("\nDebug - Unit Positions:")
        for zergling in self.zerglings:
            print(f"Zergling: x={zergling.x}, y={zergling.y}")
            display_grid[zergling.y][zergling.x] = ' Z'

        for marine in self.marines:
            print(f"Marine: x={marine.x}, y={marine.y}")
            display_grid[marine.y][marine.x] = ' M'

        # Print the board
        for row in display_grid:
            print(''.join(row))

        # Print additional unit information
        print(f"\nMarines: {len(self.marines)}, Zerglings: {len(self.zerglings)}")




In [47]:
class Unit:
    def __init__(self, x, y, hp, speed, damage, attack_range, team):
        self.x = x
        self.y = y
        self.hp = hp
        self.speed = speed
        self.damage = damage
        self.attack_range = attack_range
        self.team = team  # 'Marine' or 'Zerg'
        self.engaged_in_melee = False



class Marine(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (9.8)
        super().__init__(x, y, hp=45, speed=3, damage=9.8, attack_range=5, team='Marine')


class Zergling(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (10)
        super().__init__(x, y, hp=35, speed=4, damage=10, attack_range=1, team='Zergling')



In [48]:
# Max ply depth into the constructor,
# seperate the evaluation and the game over conditions

class SearchTreeNode:
    def __init__(self, board_instance, current_team, ply=0):
        self.children = []
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.current_team = current_team

        if not self.is_game_over() and self.ply_depth < 3:
            self.generate_children()

    def is_game_over(self):
        # Check if either team is completely eliminated
        return (len(self.current_board.marines) == 0 or
                len(self.current_board.zerglings) == 0)

    def get_units_by_center_distance(self):
        # Sort units by distance from board center
        def distance_from_center(unit):
            return abs(unit.x - self.current_board.center_x) + abs(unit.y - self.current_board.center_y)

        units = self.current_board.marines + self.current_board.zerglings
        return sorted(units, key=distance_from_center)

    def generate_children(self):
        units = self.get_units_by_center_distance()
        current_team_units = [u for u in units if u.team == self.current_team]

        for unit in current_team_units:
            possible_actions = self.current_board.get_possible_actions(unit)

            for action_type, target in possible_actions:
                # Create a deep copy of the current board to preserve state
                new_board = self.clone_board()

                if action_type == 'move':
                    self.move_unit(new_board, unit, target[0], target[1])
                else:  # attack
                    self.perform_attack(new_board, unit, target)

                self.children.append(SearchTreeNode(new_board,'Marine' if self.current_team == 'Zergling' else 'Zergling',self.ply_depth + 1))

    def clone_board(self):
        # Deep copy method for the board
        new_board = CurrentBoard(width=self.current_board.width,height=self.current_board.height)

        # Clone marines
        for marine in self.current_board.marines:
            new_marine = Marine(marine.x, marine.y)
            new_marine.hp = marine.hp
            new_board.marines.append(new_marine)
            new_board.board[new_marine.y][new_marine.x] = new_marine

        # Clone zerglings
        for zergling in self.current_board.zerglings:
            new_zergling = Zergling(zergling.x, zergling.y)
            new_zergling.hp = zergling.hp
            new_board.zerglings.append(new_zergling)
            new_board.board[new_zergling.y][new_zergling.x] = new_zergling

        return new_board

    def move_unit(self, board, unit, new_x, new_y):
        # Remove unit from old position
        board.board[unit.y][unit.x] = None

        # Update unit's position
        unit.x = new_x
        unit.y = new_y

        # Place unit in new position
        board.board[new_y][new_x] = unit

    def perform_attack(self, board, attacker, target):
        # Find the target in the board's unit lists
        if attacker.team == 'Marine':
            target_list = board.zerglings
        else:
            target_list = board.marines

        # Apply damage
        target.hp -= attacker.damage

        # Remove target if destroyed
        if target.hp <= 0:
            board.board[target.y][target.x] = None
            target_list.remove(target)

    def evaluate_tactical_position(self, team):
        score = 0
        if team == 'Marine':
            for marine in self.current_board.marines:
                # Distance from zerglings
                for zerg in self.current_board.zerglings:
                    distance = abs(marine.x - zerg.x) + abs(marine.y - zerg.y)
                    if distance == marine.attack_range:
                        score += 10  # Optimal range
                    elif distance < marine.attack_range:
                        score -= 15  # Too close to zerglings

                # Group cohesion with other marines
                for other_marine in self.current_board.marines:
                    if marine != other_marine:
                        distance = abs(marine.x - other_marine.x) + abs(marine.y - other_marine.y)
                        if 2 <= distance <= 3:
                            score += 5  # Good supporting distance
                        elif distance < 2:
                            score -= 2  # Too clustered

                # Bonus for being in a defensive position (back half of the board)
                if marine.x > self.current_board.width * 0.6:
                    score += 10

        else:  # Zergling
            for zergling in self.current_board.zerglings:
                # Distance to closest marine
                closest_marine_dist = float('inf')
                for marine in self.current_board.marines:
                    distance = abs(zergling.x - marine.x) + abs(zergling.y - marine.y)
                    closest_marine_dist = min(closest_marine_dist, distance)

                    if distance <= 1:  # In melee range
                        score += 20
                    elif distance <= 3:  # Close enough to threaten
                        score += 10

                # Surround bonus
                for marine in self.current_board.marines:
                    attacking_directions = set()
                    if abs(zergling.x - marine.x) <= 1 and abs(zergling.y - marine.y) <= 1:
                        direction = (
                            1 if zergling.x > marine.x else -1 if zergling.x < marine.x else 0,
                            1 if zergling.y > marine.y else -1 if zergling.y < marine.y else 0
                        )
                        attacking_directions.add(direction)
                        if len(attacking_directions) > 1:  # Surrounding bonus
                            score += 5

                # Bonus for being in the front half of the board for Zerglings
                if zergling.x < self.current_board.width * 0.4:
                    score += 10

        return score

    def evaluate_position(self):
        # Evaluation considering:
        # 1. Total HP of each team
        # 2. Tactical positioning
        # 3. Unit count
        marine_score = sum([m.hp for m in self.current_board.marines])
        zerg_score = sum([z.hp for z in self.current_board.zerglings])

        # Add position scoring from tactical evaluation
        marine_position = self.evaluate_tactical_position('Marine')
        zerg_position = self.evaluate_tactical_position('Zergling')

        return (marine_score + marine_position) - (zerg_score + zerg_position)

    def minimax_with_alpha_beta(self, depth, alpha, beta, is_maximizing):
        # Base case: reached depth limit or game over
        if depth == 0 or self.is_game_over():
            return self.evaluate_position()

        if is_maximizing:
            max_eval = float('-inf')
            for child in self.children:
                eval = child.minimax_with_alpha_beta(depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cutoff
            return max_eval
        else:
            min_eval = float('inf')
            for child in self.children:
                eval = child.minimax_with_alpha_beta(depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cutoff
            return min_eval

    def get_best_move(self, depth=3):
        best_value = float('-inf')
        best_move = None

        for child in self.children:
            move_value = child.minimax_with_alpha_beta(depth, float('-inf'), float('inf'), False)
            if move_value > best_value:
                best_value = move_value
                best_move = child

        return best_move


In [49]:
# Create a board
board = CurrentBoard(width=30, height=10,zergling_count=8)

# Initial setup will be automatic by calling setup_initial_board()
board.setup_initial_board()

# Display the initial board state
board.display_board()

# was having issues with the board set up, display and debug helped to show this better.



Debug - Unit Positions:
Zergling: x=0, y=3
Zergling: x=0, y=4
Zergling: x=0, y=5
Zergling: x=0, y=6
Zergling: x=1, y=3
Zergling: x=1, y=4
Zergling: x=1, y=5
Zergling: x=1, y=6
Marine: x=29, y=4
Marine: x=29, y=5
Marine: x=28, y=4
Marine: x=28, y=5
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 4, Zerglings: 8
